### **[미션] 나만의 FAQ 봇 만들기**
앞에서 실습해 본 `예제 2: 간단한 질문-답변 검색 시스템 (FAQ 봇)`에 자신만의 적절한 데이터를 수집하고 반영하여 OOO 봇을 만들어 보세요.

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1) 한국어 임베딩 모델 로드
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 2) FAQ 데이터베이스 (게임 추천 예시)
faq_database = {
    "questions": [
        "힐링되는 게임 추천해줘",
        "커플이 같이 하기 좋은 협동 게임 뭐가 있나요?",
        "닌텐도 스위치에서 파티 게임 추천",
        "스토리 몰입감 높은 싱글 플레이 게임 추천",
        "인디 로그라이크 게임 뭐가 재밌나요?",
        "전략/경영 시뮬레이션 게임 추천",
        "빠르게 한 판 즐길 FPS 멀티 게임 추천",
        "레이싱 게임 추천해줘",
        "플레이스테이션에서만 즐길 명작 추천",
        "모바일로 가볍게 즐길만한 게임 추천",
        "소울라이크 난이도 높은 게임 추천",
        "카우치 코옵(로컬) 게임 추천"
    ],
    "answers": [
        # 힐링
        "힐링/농사 계열: 『스타듀 밸리』, 『코랄 아일랜드』, 『동물의 숲』. 편안한 진행과 수집·꾸미기가 강점.",
        # 커플 협동
        "커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.",
        # 스위치 파티
        "스위치 파티: 『마리오 파티 슈퍼스타즈』, 『마리오 카트 8 디럭스』, 『스플래툰 3』. 가족/친구와 가볍게 즐기기 좋아요.",
        # 스토리 싱글
        "싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.",
        # 로그라이크
        "인디 로그라이크: 『Hades』, 『Dead Cells』, 『Slay the Spire』. 반복 도전과 빌드 다양성이 장점.",
        # 전략/경영
        "전략/경영: 『문명 6』, 『스텔라리스』, 『시티즈: 스카이라인 II』. 장기 전략·경영을 좋아하면 추천.",
        # FPS 멀티
        "FPS 멀티: 『오버워치 2』, 『에이펙스 레전드』, 『VALORANT』. 빠른 템포의 팀 기반 대전.",
        # 레이싱
        "레이싱: 『Forza Horizon 5』, 『그란 투리스모 7』, 『마리오 카트 8 디럭스』. 아케이드~시뮬 밸런스.",
        # PS 독점계열/강점
        "PS 추천: 『갓 오브 워(2018/라그나로크)』, 『라스트 오브 어스』, 『고스트 오브 쓰시마』. 연출·서사·손맛 우수.",
        # 모바일 캐주얼
        "모바일 캐주얼: 『브롤스타즈』, 『하스스톤』, 『다운웰』. 짧은 플레이 세션에 적합.",
        # 소울라이크
        "소울라이크: 『엘든 링』, 『세키로: 섀도우 다이 트와이스』, 『다크 소울 3』. 높은 난이도·전투 완성도.",
        # 카우치 코옵
        "카우치 코옵: 『오버쿡드! 2』, 『무브 오어 다이』, 『원더보이: 더 드래곤즈 트랩(교대 플레이)』 등 소파에서 함께 즐기기 좋아요."
    ]
}

# 3) FAQ 임베딩 사전 계산
print("\nFAQ 데이터베이스를 임베딩으로 변환 중...")
faq_embeddings = model.encode(faq_database["questions"])
print(f"총 {len(faq_database['questions'])}개의 FAQ가 준비되었습니다.")
print("-" * 70, '\n')

# 4) 사용자 질문 처리 함수 (예제 스타일 유지)
def find_answer(user_question, top_k=3, threshold=0.55):
    """
    사용자 질문에 가장 유사한 FAQ를 찾아 답변을 반환
    - user_question: 문자열
    - top_k: 상위 유사 질의/답변 개수
    - threshold: 최종 매칭 최소 유사도
    """
    q_emb = model.encode([user_question])
    sims = cosine_similarity(q_emb, faq_embeddings)[0]
    top_idx = np.argsort(sims)[-top_k:][::-1]

    print(f"✅ 질문: {user_question}")
    print("=" * 70)
    for rank, idx in enumerate(top_idx, 1):
        print(f"\n\t[{rank}순위] 유사도: {sims[idx]:.4f}")
        print(f"\t유사한 질문: {faq_database['questions'][idx]}")
        print(f"\t답변: {faq_database['answers'][idx]}")
        print("\t" + "-" * 70)

    best = top_idx[0]
    return faq_database["answers"][best], sims[best], (sims[best] >= threshold)

# 5) 테스트 시나리오 (5개)
test_questions = [
    "스토리 몰입감 있는 싱글 게임 뭐가 좋아요?",    # 스토리 싱글
    "남자친구랑 같이 할 협동 게임 추천해줘",         # 커플 협동
    "스위치로 파티게임 하고 싶은데 추천 있을까요?",   # 스위치 파티
    "요즘 재밌는 로그라이크 인디 추천",               # 로그라이크
    "편하게 힐링할만한 게임 알려줘"                   # 힐링
]

for test_q in test_questions:
    answer, similarity, ok = find_answer(test_q, top_k=3, threshold=0.55)
    print(f"\n\t{'='*70}")
    print(f"\t★ 최종 답변 (신뢰도: {similarity:.4f})")
    if not ok:
        print("\t⚠️ 유사도가 낮습니다. FAQ 범위를 넓히거나 질문을 더 구체화해보세요.")
    print(f"\t답변: {answer}")
    print(f"\t{'='*70}\n\n")



FAQ 데이터베이스를 임베딩으로 변환 중...
총 12개의 FAQ가 준비되었습니다.
---------------------------------------------------------------------- 

✅ 질문: 스토리 몰입감 있는 싱글 게임 뭐가 좋아요?

	[1순위] 유사도: 0.9141
	유사한 질문: 스토리 몰입감 높은 싱글 플레이 게임 추천
	답변: 싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.
	----------------------------------------------------------------------

	[2순위] 유사도: 0.5863
	유사한 질문: 인디 로그라이크 게임 뭐가 재밌나요?
	답변: 인디 로그라이크: 『Hades』, 『Dead Cells』, 『Slay the Spire』. 반복 도전과 빌드 다양성이 장점.
	----------------------------------------------------------------------

	[3순위] 유사도: 0.5416
	유사한 질문: 커플이 같이 하기 좋은 협동 게임 뭐가 있나요?
	답변: 커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.
	----------------------------------------------------------------------

	★ 최종 답변 (신뢰도: 0.9141)
	답변: 싱글 스토리: 『위쳐 3』, 『레드 데드 리뎀션 2』, 『디스코 엘리시움』. 세계관/서사가 강력.


✅ 질문: 남자친구랑 같이 할 협동 게임 추천해줘

	[1순위] 유사도: 0.7032
	유사한 질문: 커플이 같이 하기 좋은 협동 게임 뭐가 있나요?
	답변: 커플 협동: 『It Takes Two』, 『오버쿡드! 2』, 『언래벨 투』. 협동 퍼즐/조작이 재미의 핵심.
	-----------------------------------